In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

In [24]:
def read_images():
    training_images = []
    i = 0
    DIR = "E:\Galaxy Zoo\Data\images_training_rev1\images_training_rev1"
    for img in os.listdir(DIR):
        im = cv2.imread(os.path.join(DIR,img))
        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212
        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106
        training_images.append(im)
        i+=1
        if i==10000:
            break
        
    return training_images
    

In [25]:
training_images = read_images()
training_images = np.array(training_images).reshape(-1,106,106,3)

In [33]:
X_train = training_images[:8000]
x_test = training_images[8000:]
df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")
y_train = df.drop('GalaxyID', axis = 1).values
y_train = y_train[:10000]
y_test = y_train[8000:10000]
y_train = y_train[:8000]

In [27]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam

In [49]:
model = Sequential()
model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

model.add(ZeroPadding2D((1,1))) 
model.add(Conv2D((64), kernel_size = (3,3), input_shape = (106,106,3)))
model.add(Activation("relu"))

model.add(ZeroPadding2D((1,1))) 
model.add(Conv2D((64), kernel_size = (3,3), input_shape = (106,106,3)))
model.add(Activation("relu"))

model.add(ZeroPadding2D((1,1))) 
model.add(Conv2D((32), kernel_size = (3,3), input_shape = (106,106,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (3,3), strides = (3,3)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.3))


model.add(Dense(37))
model.add(Activation("sigmoid"))

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])


In [50]:
print(y_train.shape,
X_train.shape)

(8000, 37) (8000, 106, 106, 3)


In [51]:
model.fit(X_train,y_train, epochs = 50)

Epoch 1/50
8000/8000 [==============================] - 256s 32ms/step - loss: 0.2712 - acc: 0.0371
Epoch 2/50
8000/8000 [==============================] - 246s 31ms/step - loss: 0.1948 - acc: 0.0429
Epoch 3/50
8000/8000 [==============================] - 245s 31ms/step - loss: 0.1876 - acc: 0.1343
Epoch 4/50
8000/8000 [==============================] - 248s 31ms/step - loss: 0.1863 - acc: 0.1313
Epoch 5/50
8000/8000 [==============================] - 247s 31ms/step - loss: 0.1852 - acc: 0.0944
Epoch 6/50
8000/8000 [==============================] - 249s 31ms/step - loss: 0.1845 - acc: 0.0912
Epoch 7/50
8000/8000 [==============================] - 246s 31ms/step - loss: 0.1835 - acc: 0.0711
Epoch 8/50
8000/8000 [==============================] - 247s 31ms/step - loss: 0.1827 - acc: 0.0561
Epoch 9/50
8000/8000 [==============================] - 254s 32ms/step - loss: 0.1816 - acc: 0.0570
Epoch 10/50
8000/8000 [==============================] - 253s 32ms/step - loss: 0.1809 - acc: 0.0434

In [53]:
model.evaluate(x_test,y_test)

2000/2000 [==============================] - 17s 8ms/step


[0.09136274361610412, 0.024]

In [54]:
model.save("10819.model")